<a href="https://colab.research.google.com/github/chengyang122/Metter.js-and-MediaPipe-Interactive-game/blob/main/motiondetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from abc import ABC, abstractmethod
import numpy as np
import cv2
import time
import datetime
import time
# https://docs.opencv.org/3.4.2/d2/d55/group__bgsegm.html#gga1a5838fa2d2697ac455b136bfcdb4600ad8f108b7815d6bcccd32b849063e0a9c
# https://docs.opencv.org/3.4.2/db/d5c/tutorial_py_bg_subtraction.html
class DefaultAlgorithm():
    def __init__(self, instance_object):
        # print("I am in default algorithm")
        self.current_object = instance_object
        self.min_area = 0
        self.previous_motion = None
        self.current_frame = None
        self.output_frame = None
        # print(self.current_object)

    def apply(self, frame):
        self.current_frame = frame
        temp2 = frame        
        temp1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        temp2 = cv2.GaussianBlur(temp1, (11,11), 0)
        output_frame = self.current_object.apply(temp2)
        self.output_frame = output_frame
        return output_frame

    def detect_motion(self):
        total_white_pixels = np.sum(self.output_frame > 100)
        # print(total_white_pixels)
        motion = None
        if(total_white_pixels > 0.05*self.output_frame.shape[0]*self.output_frame.shape[1]):
            motion = True
        else:           
            motion = False
        return self.motion_decision(motion)

    def motion_decision(self, motion):
        if self.previous_motion is None:
            self.previous_motion = motion
            return
        if self.previous_motion != motion:
            self.previous_motion = motion
            # print(motion)
        return


        

class BackgroundSubtractorCNT(DefaultAlgorithm):
    #time : 5.1 for test02.mp4 -> laptop
    def __init__(self):
        minPixelStability = 30
        useHistory = False
        maxPixelStability = 60
        isParallel = False
        bs_cnt = cv2.bgsegm.createBackgroundSubtractorCNT(minPixelStability, useHistory, maxPixelStability, isParallel)
        # bscnt = np.zeros([3,4])
        # print("I am in BackgroundSubtractorCNT")
        super().__init__(bs_cnt)
    
    def apply(self, frame):
        return super().apply(frame)

class BackgroundSubtractorGMG(DefaultAlgorithm):
    #time : 16.8 for test02.mp4 -> laptop
    def __init__(self):
        initializationFrames = 10
        decisionThreshold = 0.8
        bs_gng = cv2.bgsegm.createBackgroundSubtractorGMG(initializationFrames, decisionThreshold)
        super().__init__(bs_gng)

    def apply(self, frame):
        return super().apply(frame)

class BackgroundSubtractorGSOC(DefaultAlgorithm):
    #time : 25.21 for test02 -> laptop
    def __init__(self):
        mc = cv2.bgsegm.LSBP_CAMERA_MOTION_COMPENSATION_NONE
        nSamples = 20
        replaceRate = 0.003
        propagationRate = 0.01
        hitsThreshold = 32
        alpha = 0.01
        beta = 0.0022
        blinkingSupressionDecay = 0.1
        blinkingSupressionMultiplier = 0.1
        noiseRemovalThresholdFacBG = 0.0004
        noiseRemovalThresholdFacFG = 0.0008
        bs_gsoc = cv2.bgsegm.createBackgroundSubtractorGSOC(mc, nSamples, replaceRate, propagationRate, hitsThreshold, alpha, beta, blinkingSupressionDecay, 
                                                            blinkingSupressionMultiplier, noiseRemovalThresholdFacBG, noiseRemovalThresholdFacFG)
        super().__init__(bs_gsoc)

    def apply(self, frame):
        return super().apply(frame)

class BackgroundSubtractorMOG(DefaultAlgorithm):
    #time : 10.91 for test02 -> laptop
    def __init__(self):
        history = 200
        nmixtures = 5
        backgroundRatio = 0.7
        noiseSigma = 0
        bs_mog = cv2.bgsegm.createBackgroundSubtractorMOG()
        super().__init__(bs_mog)

    def apply(self, frame):
        return super().apply(frame)

class BackgroundSubtractorLSBP(DefaultAlgorithm):
    #time: 44.05 for test02.mp4 -> laptop
    def __init__(self):
        mc = cv2.bgsegm.LSBP_CAMERA_MOTION_COMPENSATION_NONE
        nSamples = 20
        LSBPRadius = 16
        Tlower = 2.0
        Tupper = 32.0
        Tinc = 1.0
        Tdec = 0.05
        Rscale = 10.0
        Rincdec = 0.005
        noiseRemovalThresholdFacBG = 0.0004
        noiseRemovalThresholdFacFG = 0.0008
        LSBPthreshold = 8
        mincount = 2
        bs_lsbp = cv2.bgsegm.createBackgroundSubtractorLSBP(mc, nSamples, LSBPRadius, Tlower, Tupper, Tinc, Tdec, Rscale, Rincdec,   \
                                                            noiseRemovalThresholdFacBG, noiseRemovalThresholdFacFG, LSBPthreshold,   \
                                                            mincount)
        super().__init__(bs_lsbp)

    def apply(self, frame):
        return super().apply(frame)   

class MotionSaliency(DefaultAlgorithm):
    def __init__(self):
        self.motion_saliency = cv2.saliency.MotionSaliencyBinWangApr2014_create()

    def apply(self, frame):
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        output_frame = self.motion_saliency.computeSaliency(gray_frame)        
        print("I ma here")
        print(type(output_frame))
        return output_frame

class BackgroundSubtractorGaussianBlur(DefaultAlgorithm):
    def __init__(self):
        self.first_frame = None

    def apply(self, frame):
        motion = False
        temp1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        temp2 = cv2.GaussianBlur(temp1, (21,21), 0)
        if self.first_frame is None:
            self.first_frame = temp2
            return self.first_frame
        frame_delta = cv2.absdiff(self.first_frame, temp2)
        threshold_frame = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]
        temp1 = cv2.dilate(threshold_frame, None, iterations=2)
        (_,cnts, _) = cv2.findContours(temp1, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_SIMPLE)
        for c in cnts:
            if cv2.contourArea(c) < 500:
                motion = False
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            motion = True
        cv2.putText(frame, "Motion: {}".format(motion), (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, ((255, 150, 0), (0, 0, 255))[motion], 2)
        cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
                    (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255, 150, 0), 1)
        return frame_delta



def main():
    start_time = time.time()
    cap = cv2.VideoCapture(0)
    motion_detection_algo = BackgroundSubtractorCNT()
    # ret, input_frame = cap.read()
    # output_frame = motion_detection_algo.apply(input_frame)
    # print(type(output_frame))
    stacklist = []
    while(True):
        # Capture frame-by-frame
        ret, input_frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            break
        output_frame = motion_detection_algo.apply(input_frame)
        # motion_detection_algo.detect_motion()
        cv2.imshow('input_frame',input_frame)
        #cv2.imshow('output_frame', output_frame)
        newest_average = np.mean(output_frame)
        stacklist.append(newest_average)
        if len(stacklist)>10:
          stacklist.pop(0)
        if np.sum(stacklist[1:])<0.1 and stacklist[0]>0:
          upload_frame = inputframe
        cv2.imshow('upload_frame', upload_frame)
        switch = cv2.waitKey(1) & 0xFF
        if switch == ord('q'):
            break
    cv2.destroyAllWindows()
    cap.release()
    print("%s sec"%(time.time()-start_time))

if __name__=="__main__":
    main()
